# Sales Analysis For Northwind Traders' Operations

Northwind Traders is a food distribution company. Its history of operations is tracked in its [database found here](https://github.com/pthom/northwind_psql/tree/master) and includes information relating to four key areas: employee performance, monthly and yearly sales data and growth, consumer purchasing behaviour, and product sales and categories.

The purpose of this project is to analyze these key areas to help Northwind Traders' management make strong, data-backed decisions and answer the following questions:

1. Which employees should be rewarded/provided support based on performance, and how can rankings promote healthy competition and better sales?
2. What are monthly and yearly sales growth trends?
3. How can the highest-spending and lowest-spending customers be identified to determine appropriate engagement incentives?
4. What are the top products and how can inventory for these be optimized?
    
After running the data analysis, I found the top performing employees by sales rank; Northwind Traders' running total of sales and month-over-month sales growth rate; engagement levels of customers' orders, ranked by percentile; and the highest performing categories, individual products within each category, and individual products overall. All tables summarizing this data will prove invaluable for Northwind Traders to optimize its operations and continue strong growth into the future with data-backed decisions and business strategies.


### Employee Sales Performance

To answer the first question, I join the `employees` table with the `orders` and `order_details` tables to link each employee with their respective sales performance. I defined each employee's total sales as the sum of the revenue from all the orders they handle, and determined this first with an `employee_sales` Common Table Expression (CTE). This allowed me to then query the employees' data and rank them by their total sales.

In [ ]:
WITH
employee_sales AS (
    SELECT e.employee_id, e.first_name, e.last_name, 
           ROUND(SUM(od.quantity * od.unit_price)::numeric, 2) AS total_sales
      FROM employees AS e
      JOIN orders AS o
        ON o.employee_id = e.employee_id
      JOIN order_details AS od
        ON od.order_id = o.order_id
     GROUP BY e.employee_id
)

SELECT employee_id, first_name || ' ' || last_name AS full_name, total_sales,
       RANK() OVER(ORDER BY total_sales DESC) AS rank
  FROM employee_sales;

| employee_id | full_name | total_sales | rank |
|:--------:|:--------:|:--------:|:--------:|
| 4 | Margaret Peacock | 250187.45 | 1 |
| 3 | Janet Leverling | 213051.3 | 2 |
| 1 | Nancy Davolio | 202143.71 | 3 |
| 2 | Andrew Fuller | 177749.26 | 4 |
| 7 | Robert King | 141295.99 | 5 |
| 8 | Laura Callahan | 133301.03 | 6 |
| 9 | Anne Dodsworth | 82964 | 7 |
| 6 | Michael Suyama | 78198.1 | 8 |
| 5 | Steven Buchanan | 75567.75 | 9 |

The top three performing employees all average over 200,000 revenue in sales, while the bottom three employees average around 80,000. To help fix the large disparity, this data can be used by management to reward employees such as Margaret Peacock, Janet Leverling, and Nancy Davolio to foster a company culture of strong performance and engage the employees, while distributing additional resources and support to help employees like Anne Dodsworth, Michael Suyama, and Steven Buchanan significantly improve their sales over time.

Overall, the ranking above provides management valuable data to promote healthy competition within the company to ultimately boost sales and help employees develop.

### Sales Trends: Running Total & Month-Over-Month Growth

The employees' sales performances provide a snapshot of individual contribution, but examining Northwind Traders' macro sales trends will provide more valuable data on sales growth rates. I begin by querying a table for sales by month and keeping a running total of monthly sales over time with an aggregate window function.

In [ ]:
WITH
monthly_sales AS (
    SELECT EXTRACT(month FROM o.order_date) AS month,
           ROUND(SUM(od.unit_price * od.quantity)::numeric, 2) AS total_sale
      FROM order_details AS od
      JOIN orders AS o
        ON od.order_id = o.order_id
     GROUP BY EXTRACT(month FROM o.order_date)
)

SELECT month, total_sale,
       ROUND(SUM(total_sale) OVER(ORDER BY month)::numeric, 2) AS running_total
  FROM monthly_sales
 ORDER BY month;

| month | total_sales | running_total |
|:--------:|:--------:|:--------:|
| 1 | 167547.52 | 167547.52
| 2 | 145769.15 | 313316.67
3 | 149805.35 | 463122.02
4 | 190329.95 | 653451.97
5 | 76722.36 | 730174.33
6 | 39088 | 769262.33
7 | 85657.03 | 854919.36
8 | 76591.09 | 931510.45
9 | 87369.02 | 1018879.47
10 | 111532.1 | 1130411.57
11 | 95617.36 | 1226028.93
12 | 128429.66 | 1354458.59


The running total of sales by month shows the total sales each month but does not fully capture how sales have changed relative to the previous month. To generate this data, I can query a table displaying the month-over-month sales growth rate. This can be done with CTEs and the window offset function `LAG()` to query the previous month's sales and compare it with the current month's.

In [ ]:
WITH
monthly_sales AS (
    SELECT EXTRACT(YEAR FROM o.order_date) AS year,
           EXTRACT(MONTH FROM o.order_date) AS month,
           ROUND(SUM(od.unit_price * od.quantity)::numeric, 2) AS total_sale
      FROM order_details AS od
      JOIN orders AS o
        ON od.order_id = o.order_id
     GROUP BY EXTRACT(YEAR FROM o.order_date), EXTRACT(MONTH FROM o.order_date)
),
lag_sales AS (
    SELECT year, month, total_sale,
           LAG(total_sale) OVER(ORDER BY year, month) AS prev_month
      FROM monthly_sales
)
    
SELECT year, month, ROUND(((total_sale - prev_month) * 100 / prev_month), 2) || '%' AS growth
  FROM lag_sales
 LIMIT 13;

| year | month | growth |
|:--------:|:--------:|:--------:|
1997 | 7 |
1996 | 8 | -11.87%
1996 | 9 | 3.86%
1996 | 10 | 49.09%
1996 | 11 | 20.63%
1996 | 12 | 2.51%
1997 | 1 | 30.89%
1997 | 2 | -38.21%
1997 | 3 | -2.98%
1997 | 4 | 39.32%
1997 | 5 | 2.02%
1997 | 6 | -31.21%
1997 | 7 | 41.90%

The month-over-month sales growth chart clearly outlines when total sales have increased or decreased relative to the preceding month: for example, the period from September 1996 to the new year saw immense sales growth before declining for two months after. Overall, Northwind Traders is on an upwards trajectory with extremely strong sales growth rates.

The growth-rate data can be further analyzed by management for patterns relating to business decisions: for example, how does the introduction of a new marketing campaign or product affect sales growth? Both sales tables I queried provide value to better inform business decisions and judge the effectiveness of certain trialed strategies.

### Customer Engagement Analysis

Northwind Traders' sales depend on customer behaviour and their engagement levels. As such, identifying which customers are high-engagers—that is, spend the most—and which ones are low-engagers will allow management to implement appropriate engagement incentives or rewards. Top customers may be drawn into a loyalty program to keep their engagement levels, while special promotions may be targeted towards lower-spending customers to retain them and ultimately increase sales.

I can determine customer engagement by querying a table using `CASE` to tag customer orders as either "Above Average", "Average", or "Below Average":

In [ ]:
WITH
customer_order_value AS (
    SELECT c.customer_id, o.order_id,
           ROUND(SUM(od.quantity * od.unit_price)::numeric, 2) AS value
      FROM customers AS c
      JOIN orders AS o
        ON c.customer_id = o.customer_id
      JOIN order_details AS od
        ON o.order_id = od.order_id
     GROUP BY c.customer_id, o.order_id
)

SELECT customer_id, order_id, value,
       CASE
       WHEN value > AVG(value) OVER() THEN 'Above Average'
       WHEN value = AVG(value) OVER() THEN 'Average'
       ELSE 'Below Average'
       END AS order_value,
       ROUND(100 * PERCENT_RANK() OVER (ORDER BY value)::numeric, 2) || 'th' AS percentile
  FROM customer_order_value
 ORDER BY value DESC;

| customer_id | order_id | value | order_value | percentile |
|:--------:|:--------:|:--------:|:--------:|:--------:|
QUICK | 10865 | 17250 | Above Average | 100.00th
SAVEA | 11030 | 16321.9 | Above Average | 99.88th
HANAR | 10981 | 15810 | Above Average | 99.76th
QUEEN | 10372 | 12281.2 | Above Average | 99.64th
MEREP | 10424 | 11493.2 | Above Average | 99.52th
KOENE | 10817 | 11490.7 | Above Average | 99.40th
RATTC | 10889 | 11380 | Above Average | 99.28th
SIMOB | 10417 | 11283.2 | Above Average | 99.16th
HUNGO | 10897 | 10835.24 | Above Average | 99.03th
PICCO | 10353 | 10741.6 | Above Average | 98.91th
QUICK | 10515 | 10588.5 | Above Average | 98.79th
RATTC | 10479 | 10495.6 | Above Average | 98.67th
QUICK | 10540 | 10191.7 | Above Average | 98.55th
QUICK | 10691 | 10164.8 | Above Average | 98.43th
WHITC | 11032 | 8902.5 | Above Average | 98.31th
   ...   |  ...     |   ...    |      ...        | ...
MAGAA | 10754 | 55.2 | Below Average | 1.69th
SPECD | 10738 | 52.35 | Below Average | 1.57th
FRANS | 10422 | 49.8 | Below Average | 1.45th
SPLIR | 10271 | 48 | Below Average | 1.33th
WELLI | 10900 | 45 | Below Average | 0.84th
ISLAT | 10674 | 45 | Below Average | 0.84th
NORTS | 11057 | 45 | Below Average | 0.84th
LAMAI | 11051 | 45 | Below Average | 0.84th
SAVEA | 10815 | 40 | Below Average | 0.72th
LONEP | 10883 | 36 | Below Average | 0.60th
OCEAN | 10898 | 30 | Below Average | 0.48th
REGGC | 10586 | 28 | Below Average | 0.24th
SUPRD | 10767 | 28 | Below Average | 0.24th
FRANS | 10807 | 18.4 | Below Average | 0.12th
CACTU | 10782 | 12.5 | Below Average | 0.00th

The table indicates customer orders based on a general categorization of how they compare to the order average. Moreover, I queried each customer order's percentile, which will be useful should Northwind Traders require more specific ranking information regaring how much value each customer brings in relative to other customers. For example, using the percentile ranking will allow Northwind Traders to target a rewards program to all customers with orders above the 90th percentile and a marketing offer campaign to customers within the lower 50th percentile to encourage engagement.

### Product and Category Analysis

Building off the customer analysis in the previous section, it is equally important to consider which products and product categories are the biggest drivers of Northwind Traders' sales and revenue. Data showing the most popular products and sales compositions within each category will allow the company to optimize its operations. Such can be accomplished by streamlining inventory to meet demand and deciding methods of boosting currently sub-optimal product categories. Product and category analysis data will also allow for decisions to be made regarding what new product and category offerings are most likely to be successful.

I begin by querying the percentage of total sales that each product category takes up. Using a CTE to calculate sales by category and aggregate window funtions, I can determine the sales categories that make up the biggest percentage of the company's overall sales.


In [ ]:
WITH
sales_by_category AS (
    SELECT c.category_id, c.category_name,
           ROUND(SUM(od.unit_price * od.quantity)::numeric, 2) AS total_category_sales
      FROM categories AS c
      JOIN products AS p
        ON c.category_id = p.category_id
      JOIN order_details AS od
        ON p.product_id = od.product_id
     GROUP BY c.category_id
     ORDER BY category_id, category_name
)

SELECT category_id, category_name, total_category_sales,
       ROUND(total_category_sales / SUM(total_category_sales) OVER() * 100, 2)
       || '%' AS percentage
  FROM sales_by_category
 GROUP BY category_id, category_name, total_category_sales;



category_id | category_name | total_category_sales | percentage
|:--------:|:--------:|:--------:|:--------:|
1 | Beverages | 286526.95 | 21.15%
2 | Condiments | 113694.75 | 8.39%
3 | Confections | 177099.1 | 13.08%
4 | Dairy Products | 251330.5 | 18.56%
5 | Grains/Cereals | 100726.8 | 7.44%
6 | Meat/Poultry | 178188.8 | 13.16%
7 | Produce | 105268.6 | 7.77%
8 | Seafood | 141623.09 | 10.46%


The largest contributing categories to sales are `Beverages`, `Dairy Products`, and `Meat/Poultry`, collectively accounting for over 50% of all sales. 

It is important for Northwind Traders to always closely monitor products in these categories to ensure that they are available in stock to meet demand. To answer what specific products in each category are most popular, I can assign each product a rank with the `ROW_NUMBER()` window function and filter out everything except the top three most popular products from each category.

In [ ]:
WITH
product_sales AS (
    SELECT p.product_id, p.product_name, p.category_id, c.category_name,
           ROUND(SUM(od.quantity * od.unit_price)::numeric, 2) AS sales
      FROM products AS p
      JOIN order_details AS od
        ON p.product_id = od.product_id
      JOIN categories AS c
        ON p.category_id = c.category_id
     GROUP BY p.product_id, c.category_name
     ORDER BY sales DESC
)

SELECT product_id, product_name, category_id, category_name, sales, ranking AS category_rank
  FROM (
       SELECT product_id, product_name, category_id, category_name, sales,
       ROW_NUMBER() OVER(PARTITION BY category_id
                         ORDER BY sales DESC) AS ranking
         FROM product_sales
       ) AS ranking_table
 WHERE ranking <= 3;


| product_id | product_name | category_id | category_name | sales | category_rank |
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
38 | Côte de Blaye | 1 | Beverages | 149984.2 | 1
43 | Ipoh Coffee | 1 | Beverages | 25079.2 | 2 
2 | Chang | 1 | Beverages | 18559.2 | 3
63 | Vegie-spread | 2 | Condiments | 17696.3 | 1 
61 | Sirop d'érable | 2 | Condiments | 16438.8 | 2
65 | Louisiana Fiery Hot Pepper Sauce | 2 | Condiments | 14607 | 3
62 | Tarte au sucre | 3 | Confections | 49827.9 | 1 
20 | Sir Rodney's Marmalade | 3 | Confections | 23635.8 | 2
26 | Gumbär Gummibärchen | 3 | Confections | 21534.9 | 3
59 | Raclette Courdavault | 4 | Dairy Products | 76296 | 1
60 | Camembert Pierrot | 4 | Dairy Products | 50286 | 2
72 | Mozzarella di Giovanni | 4 | Dairy Products | 25738.8 | 3
56 | Gnocchi di nonna Alice | 5 | Grains/Cereals | 45121.2 | 1
64 | Wimmers gute Semmelknödel | 5 | Grains/Cereals | 23009 | 2
42 | Singaporean Hokkien Fried Mee | 5 | Grains/Cereals | 9332.4 | 3
29 | Thüringer Rostbratwurst | 6 | Meat/Poultry | 87736.4 | 1
17 | Alice Mutton | 6 | Meat/Poultry | 35482.2 | 2
53 | Perth Pasties | 6 | Meat/Poultry | 21510.2 | 3
51 | Manjimup Dried Apples | 7 | Produce | 44742.6 | 1
28 | Rössle Sauerkraut | 7 | Produce | 26865.6 | 2
7 | Uncle Bob's Organic Dried Pears | 7 | Produce | 22464 | 3
18 | Carnarvon Tigers | 8 | Seafood | 31987.5 | 1
10 | Ikura | 8 | Seafood | 22140.2 | 2
40 | Boston Crab Meat | 8 | Seafood | 19048.3 | 3



The top three most popular products in each category provide management data on what type of products are essential to a category's performance and how inventory should be optimized for these high-demand products. However, what are the most popular products overall? I can answer this question by tabulating the total sales for all products across all categories and assigning each a percentile ranking, providing even more specific data on individual product performance.


In [ ]:
WITH
product_sales AS (
    SELECT p.product_id, p.product_name, p.category_id, c.category_name,
           ROUND(SUM(od.quantity * od.unit_price)::numeric, 2) AS sales
      FROM products AS p
      JOIN order_details AS od
        ON p.product_id = od.product_id
      JOIN categories AS c
        ON p.category_id = c.category_id
     GROUP BY p.product_id, c.category_name
     ORDER BY sales DESC
)

SELECT product_id, product_name, category_id, category_name, sales,
       ROUND(100 * PERCENT_RANK() OVER(ORDER BY sales)::numeric, 2) || 'th' AS percentile
  FROM (
       SELECT product_id, product_name, category_id, category_name, sales
         FROM product_sales
        ) AS ranking_table
  ORDER BY sales DESC;

product_id | product_name | category_id | category_name | sales | percentile
|:--------:|:--------:|:--------:|:--------:|:--------:|:--------:|
38 | Côte de Blaye | 1 | Beverages | 149984.2 | 100.00th
29 | Thüringer Rostbratwurst | 6 | Meat/Poultry | 87736.4 | 98.68th
59 | Raclette Courdavault | 4 | Dairy Products | 76296 | 97.37th
60 | Camembert Pierrot | 4 | Dairy Products | 50286 | 96.05th
62 | Tarte au sucre | 3 | Confections | 49827.9 | 94.74th
56 | Gnocchi di nonna Alice | 5 | Grains/Cereals | 45121.2 | 93.42th
51 | Manjimup Dried Apples | 7 | Produce | 44742.6 | 92.11th
17 | Alice Mutton | 6 | Meat/Poultry | 35482.2 | 90.79th
18 | Carnarvon Tigers | 8 | Seafood | 31987.5 | 89.47th
28 | Rössle Sauerkraut | 7 | Produce | 26865.6 | 88.16th
72 | Mozzarella di Giovanni | 4 | Dairy Products | 25738.8 | 86.84th
43 | Ipoh Coffee | 1 | Beverages | 25079.2 | 85.53th
69 | Gudbrandsdalsost | 4 | Dairy Products | 24307.2 | 84.21th
20 | Sir Rodney's Marmalade | 3 | Confections | 23635.8 | 82.89th
64 | Wimmers gute Semmelknödel | 5 | Grains/Cereals | 23009 | 81.58th
7 | Uncle Bob's Organic Dried Pears | 7 | Produce | 22464 | 80.26th
10 | Ikura | 8 | Seafood | 22140.2 | 78.95th
26 | Gumbär Gummibärchen | 3 | Confections | 21534.9 | 77.63th
53 | Perth Pasties | 6 | Meat/Poultry | 21510.2 | 76.32th
... | ... | ... | ... | ... | ...
5 | Chef Anton's Gumbo Mix | 2 | Condiments | 5801.15 | 23.68th
13 | Konbu | 8 | Seafood | 5234.4 | 22.37th
54 | Tourtière | 6 | Meat/Poultry | 5121 | 21.05th
23 | Tunnbröd | 5 | Grains/Cereals | 4840.2 | 19.74th
24 | Guaraná Fantástica | 1 | Beverages | 4782.6 | 18.42th
45 | Rogede sild | 8 | Seafood | 4740.5 | 17.11th
47 | Zaanse koeken | 3 | Confections | 4358.6 | 15.79th
73 | Röd Kaviar | 8 | Seafood | 4200 | 14.47th
25 | NuNuCa Nuß-Nougat-Creme | 3 | Confections | 4051.6 | 13.16th
66 | Louisiana Hot Spiced Okra | 2 | Condiments | 3519 | 11.84th
50 | Valkoinen suklaa | 3 | Confections | 3510 | 10.53th
52 | Filo Mix | 5 | Grains/Cereals | 3383.8 | 9.21th
3 | Aniseed Syrup | 2 | Condiments | 3080 | 7.89th
37 | Gravad lax | 8 | Seafood | 3047.2 | 6.58th
74 | Longlife Tofu | 7 | Produce | 2566 | 5.26th
67 | Laughing Lumberjack Lager | 1 | Beverages | 2562 | 3.95th
15 | Genen Shouyu | 2 | Condiments | 1813.5 | 2.63th
33 | Geitost | 4 | Dairy Products | 1713.5 | 1.32th
48 | Chocolade | 3 | Confections | 1542.75 | 0.00th

The most popular products that Northwind Traders sells are listed above. This table provides a more macro view of product performance rather than limiting them to just their categories. Using the percentile ranking, the company will also be able to quantitively identify strong and poor performers. This will allow for constant re-stocking for the top percentile products, and initiatives such as discounts on the bottom 20th percentile products to sell them out before their expiration dates.

## Conclusion

I answered the four questions posed at the beginning of the project to help Northwind Traders streamline its operations. I ranked employees by their sales performance to help management deduce ways of promoting healthy competition. I also took on a more macro view of sales performance by tabulating the running total of sales and month-by-month sales growth rate, allowing for easy identification of macro trends. I then ordered customers by percentile of their order contributions to sales, making it simple for management to engage, reward, and retain customers, and took a deep look at specific products and categories that perform the strongest to ensure their inventory optimization.

Ultimately, all of the data will allow Northwind Traders to continue making strong, data-backed decisions for a long-term strategy that helps continues its history of growth.